In [1]:
!pip install jupyter opencv-python pytesseract Pillow

     -------------------------------------- 39.5/39.5 MB 576.9 kB/s eta 0:00:00


In [36]:
!pip install easyocr

  Using cached easyocr-1.7.2-py3-none-any.whl (2.9 MB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
  Using cached torchvision-0.22.0-cp310-cp310-win_amd64.whl (1.7 MB)
  Using cached ninja-1.11.1.4-py3-none-win_amd64.whl (296 kB)
  Using cached torch-2.7.0-cp310-cp310-win_amd64.whl (212.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Admin\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [24]:
# import pytesseract

# # Replace with the actual path to your tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Windows example
# # On macOS/Linux, it might be something like:
# # pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'

# try:
#     # Now you can use pytesseract functions
#     text = pytesseract.image_to_string('your_image.png')
#     print(text)
# except pytesseract.TesseractNotFoundError:
#     print("Error: Tesseract is not installed or not found in your system's PATH.")
#     print("Please make sure Tesseract is installed and its location is correctly specified.")

Error: Tesseract is not installed or not found in your system's PATH.
Please make sure Tesseract is installed and its location is correctly specified.


In [ ]:
# !tesseract --version
# !where tesseract


In [34]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
# import pytesseract
import re
from PIL import Image
from typing import Dict, List, Any, Tuple, Optional
import pandas as pd
import easyocr


ModuleNotFoundError: No module named 'easyocr'

In [11]:
def preprocess_image(image):
    """
    Preprocess the image to improve OCR accuracy
    """
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()    
    adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    denoised = cv2.fastNlMeansDenoising(adaptive_thresh, h=10)
    
    return denoised

In [9]:
def display_image(image, title='Image'):
    plt.figure(figsize=(10, 8))
    if len(image.shape) == 3:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

<!-- def extract_text_from_image(image_path):
    """
    Extract text from an image file
    """
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at {image_path}")
    
    # Preprocess
    processed_img = preprocess_image(img)
    
    # Display original and processed images
    display_image(img, 'Original Image')
    display_image(processed_img, 'Processed Image')
    
    # Extract text using Tesseract
    text = pytesseract.image_to_string(processed_img)
    
    # Extract data with layout information
    data = pytesseract.image_to_data(processed_img, output_type=pytesseract.Output.DICT)
    
    return text, data, img, processed_img
 -->

In [29]:
def extractText(image_path):
    """
    Extract text from an image file using EasyOCR
    """
    reader = easyocr.Reader(['en'])
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at {image_path}")
    results = reader.readtext(img)
    text = " ".join([result[1] for result in results])    
    return text

In [30]:
def parse_labTexts(text:str) -> List[Dict[str, Any]]:
    print("Extractd Text:")
    print(text)
    lab_tests = []
    patterns = [
        r'([A-Z][A-Za-z\s\(\)]+)[\s:]+(\d+\.?\d*)\s*([a-zA-Z%/]+)[\s\(]+(\d+\.?\d*[-–]\d+\.?\d*)',
        r'([A-Z][A-Za-z\s\(\)]+)\s+(\d+\.?\d*)\s+([a-zA-Z%/]+)\s+(\d+\.?\d*[-–]\d+\.?\d*)'
    ]
    
    all_matches = []
    for pattern in patterns:
        matches = re.findall(pattern, text)
        all_matches.extend(matches)
    print(f"Found {len(all_matches)} potential lab test")
    
    for match in all_matches:
        if len(match) >= 4:
            test_name, test_value, test_unit, ref_range = match
            test_name = test_name.strip()
            test_value = test_values.strip()
            test_unit = test_unit.strip()
            ref_range = ref_range.strip().replace('-', '-')
            out_of_range = is_out_of_range(test_value, ref_range)
            
            lab_test = {
                "test_name" : test_name,
                "test_value" : test_value,
                "bio_reference_range" : ref_range,
                "test_unit" : test_unit,
                "lab_test_out_pf_range" : out_of_range
            }
            
            lab_test.append(lab_test)
        unique_tests = []
        test_names = set()
        
        for test in lab_tests:
            if test["test_name"] not in test_names:
                test_names.add(test["test_name"])
                unique_tests.append(test)
        return unique_tests
              

In [31]:
def is_out_of_range(value, range_str):
    """
    Check if the test value is outside the reference range
    """
    try:
        val = float(value)
        
        range_parts = range_str.replace('–', '-').split('-')
        min_val = float(range_parts[0])
        max_val = float(range_parts[1])
        
        return val < min_val or val > max_val
    except (ValueError, IndexError):
        
        print(f"Warning: Could not check range for value '{value}' and range '{range_str}'")
        return False


In [32]:
def process_image_and_extract_tests(image_path):
    """
    Process an image and extract lab tests from it
    """
    print(f"Processing image: {image_path}")
    
    text, data, original_img, processed_img = extractText(image_path)    
    lab_tests = parse_lab_tests(text)    
    result = {
        "is_success": len(lab_tests) > 0,
        "data": lab_tests
    }
    print("\nExtracted Lab Tests:")
    for i, test in enumerate(lab_tests):
        print(f"Test {i+1}:")
        print(f"  Name: {test['test_name']}")
        print(f"  Value: {test['test_value']} {test['test_unit']}")
        print(f"  Reference Range: {test['bio_reference_range']}")
        print(f"  Out of Range: {test['lab_test_out_of_range']}")
    
    return result, original_img, processed_img


In [33]:
image_folder = r'C:\Users\Admin\Downloads\bajaj_dataScience\lab_reports_samples\lbmaske'
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) 
               if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
if image_files:
    sample_image = image_files[0]
    result, original_img, processed_img = process_image_and_extract_tests(sample_image)
    
    
    import json
    print("\nJSON Output:")
    print(json.dumps(result, indent=2))



Processing image: C:\Users\Admin\Downloads\bajaj_dataScience\lab_reports_samples\lbmaske\AHD-0425-PA-0007561_JITENDRA TRIVEDI DS_28-04-2025_1019-21_AM.pdf_page_9.png


NameError: name 'easyocr' is not defined